# Movie Recommender - Item based



In [2]:
import pandas as pd
import seaborn as sns

In [55]:
path = "../ml-latest-small/"
movies = pd.read_csv(f"{path}movies.csv")
links = pd.read_csv(f"{path}links.csv")
ratings = pd.read_csv(f"{path}ratings.csv")
movies.query("movieId == 1")["title"].iloc[0]

'Toy Story (1995)'

In [11]:
ratings_dense_matrix = (
    ratings
        .drop(columns="timestamp")
        .groupby("movieId")
        .agg(mean_rating= ("rating", "mean"), count_rating= ("rating","count"))
        .reset_index()
        .assign(ratingtest = lambda x: x["mean_rating"] * (x["count_rating"] / 300))
        .round(1)
        
)
ratings_dense_matrix


,movieId,mean_rating,count_rating,ratingtest
0,1,3.9,215,2.8
1,2,3.4,110,1.3
2,3,3.3,52,0.6
3,4,2.4,7,0.1
4,5,3.1,49,0.5
...,...,...,...,...
9719,193581,4.0,1,0.0
9720,193583,3.5,1,0.0
9721,193585,3.5,1,0.0
9722,193587,3.5,1,0.0


## Recommender Function
**Input:** 
- movie_name 
- n: int = movies_to_display

**Output:** 
- top n most similar movie titles 
- based on cosine similarity

In [24]:
# py function get sparse matrix
def get_sparse_matrix(dense_matrix: pd.DataFrame): 
    dense_matrix.pivot(index='userId', columns='movieId', values=['rating'])
    return(
    dense_matrix
        .pivot(index='userId', columns='movieId', values='rating')
    )

In [56]:
# py function item based recommender
def item_based_recommender(dense_matrix: pd.DataFrame, movie_id: int, n: int =10): # n=6, minimum number of ratings

    sparse_matrix = get_sparse_matrix(ratings)
    
    movie_ids = (
    sparse_matrix
        .corrwith(sparse_matrix[movie_id])
        .sort_values(ascending=False)
        .index
        .to_list()[1:n+1]
    )
    movie_titles = [movies.query("movieId == @movie_id")["title"].iloc[0] for movie_id in movie_ids ]

    return movie_titles

item_based_recommender(ratings_dense_matrix, 1)


/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2634: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2493: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


['Package, The (1989)',
 'Gas, Food, Lodging (1992)',
 'Room in Rome (Habitación en Roma) (2010)',
 "White Man's Burden (1995)",
 'Senna (2010)',
 'Hall Pass (2011)',
 'Dead Pool, The (1988)',
 'Cedar Rapids (2011)',
 'All-Star Superman (2011)',
 'Let It Ride (1989)']